In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import six
from os import walk

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from fnmatch import fnmatch

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

from utils import label_map_util

from utils import visualization_utils as vis_util
windows = True

In [2]:
# This is needed to display the images.
%matplotlib inline

In [3]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)    

In [5]:
#pbtxt
if windows == True:
    PATH_TO_LABELS = 'E:\\TESE\\ObjectDetection\\azure_container\\Train\\teste_01\\blue_jpg\\training\\object-detection.pbtxt'
else:
    PATH_TO_LABELS = '/mnt/work/TESE/ObjectDetection/azure_container/Train/teste_01/blue_jpg/training/object-detection.pbtxt'
NUM_CLASSES = 1
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
# images
gt_121 = 648
gt_103 = 177
gt_022 = 122
gt_040 = 42
gt_085 = 57
if windows == True:
    PATH_TO_TEST_IMAGES_DIR = 'E:\\TESE\\ObjectDetection\\Data\\Images_test'
else:
    PATH_TO_TEST_IMAGES_DIR = '/mnt/work/TESE/ObjectDetection/Data/Images_test'
TEST_IMAGE_PATHS  = []
for path, subdirs, files in os.walk(PATH_TO_TEST_IMAGES_DIR):
    for name in files:
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR, name))

In [6]:
# images validation
gt_013 = 182
gt_025 = 113
gt_034 = 230
gt_043 = 73
gt_049 = 59
gt_061 = 131
gt_070 = 101
gt_076 = 154
gt_094 = 91
gt_106 = 164
gt_109 = 24
gt_118 = 661
gt_208 = 312
if windows == True:
    PATH_TO_TEST_IMAGES_DIR = 'E:\\TESE\\ObjectDetection\\Data\\Images_val'
else:
    PATH_TO_TEST_IMAGES_DIR = '/mnt/work/TESE/ObjectDetection/Data/Images_val'
validation = True
TEST_IMAGE_PATHS  = []
for path, subdirs, files in os.walk(PATH_TO_TEST_IMAGES_DIR):
    for name in files:
        TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR, name))

In [ ]:
# models
if windows == True:
    inputdir = 'E:\\TESE\\Testes\\Faster\\m05_final'
else:
    inputdir = '/mnt/work/TESE/Testes/Faster/m05_final'
pattern = '*graph.pb'

for path, subdirs, files in os.walk(inputdir):
    for name in files:
        if fnmatch(name, pattern):
            path_frozen = os.path.join(path, name)
            #frozen model
            model = path_frozen[-45:-41]
            my_df = pd.DataFrame(columns = ['0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9','GT', 'thresh'])
            PATH_TO_CKPT = path_frozen
            print("Model " + model + ' started')
            #load frozen model to memory
            detection_graph = tf.Graph()
            with detection_graph.as_default():
              od_graph_def = tf.GraphDef()
              with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            
            for path in TEST_IMAGE_PATHS:
                image_raw = Image.open(path)
                image = image_raw.resize((1100,1100), resample = Image.BILINEAR)
                image_np = load_image_into_numpy_array(image)
                image_np_expanded = np.expand_dims(image_np, axis=0)
                output_dict = run_inference_for_single_image(image_np, detection_graph)
                count_10=count_20=count_30=count_40=count_50=count_60=count_70=count_80=count_90=0
                
                for i in output_dict['detection_scores']:
                    if i > 0.10:
                        count_10 +=1
                    if i > 0.20:
                        count_20 +=1
                    if i > 0.30:
                        count_30 +=1
                    if i > 0.40:
                        count_40 +=1
                    if i > 0.50:
                        count_50 +=1
                    if i > 0.60:
                        count_60 +=1
                    if i > 0.70:
                        count_70 +=1
                    if i > 0.80:
                        count_80 +=1
                    if i > 0.90:
                        count_90 +=1
                        
                    if '121' in path:
                        thresh = output_dict['detection_scores'][648]
                        my_df.loc['Image1'] =[count_10, count_20, count_30, count_40, count_50, count_60, count_70, count_80, count_90, gt_121, thresh] 
                    elif '103' in path:
                        thresh = output_dict['detection_scores'][177]
                        my_df.loc['Image2'] =[count_10, count_20, count_30, count_40, count_50, count_60, count_70, count_80, count_90, gt_103, thresh]
                    elif '022' in path:
                        thresh = output_dict['detection_scores'][122]
                        my_df.loc['Image3'] =[count_10, count_20, count_30, count_40, count_50, count_60, count_70, count_80, count_90, gt_022, thresh]
                    elif '040' in path:
                        thresh = output_dict['detection_scores'][42]
                        my_df.loc['Image4'] =[count_10, count_20, count_30, count_40, count_50, count_60, count_70, count_80, count_90, gt_040, thresh]
                    elif '085' in path:
                        thresh = output_dict['detection_scores'][57]
                        my_df.loc['Image5'] =[count_10, count_20, count_30, count_40, count_50, count_60, count_70, count_80, count_90, gt_085, thresh]

# validation images
                    elif '013' in path:
                        thresh = output_dict['detection_scores'][182]
                        my_df.loc['Image1_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_013, thresh]
                    elif '025' in path:
                        thresh = output_dict['detection_scores'][113]
                        my_df.loc['Image2_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_025, thresh]
                    elif '034' in path:
                        thresh = output_dict['detection_scores'][230]
                        my_df.loc['Image3_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_034, thresh]
                    elif '043' in path:
                        thresh = output_dict['detection_scores'][73]
                        my_df.loc['Image4_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_043, thresh]
                    elif '049' in path:
                        thresh = output_dict['detection_scores'][59]
                        my_df.loc['Image5_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_049, thresh]
                    elif '061' in path:
                        thresh = output_dict['detection_scores'][131]
                        my_df.loc['Image6_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_061, thresh]
                    elif '070' in path:
                        thresh = output_dict['detection_scores'][101]
                        my_df.loc['Image7_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_070, thresh]
                    elif '076' in path:
                        thresh = output_dict['detection_scores'][154]
                        my_df.loc['Image8_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_076, thresh]
                    elif '094' in path:
                        thresh = output_dict['detection_scores'][91]
                        my_df.loc['Image9_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_094, thresh]
                    elif '106' in path:
                        thresh = output_dict['detection_scores'][164]
                        my_df.loc['Image10_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_106, thresh]
                    elif '109' in path:
                        thresh = output_dict['detection_scores'][23]
                        my_df.loc['Image11_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_109, thresh]
                    elif '118' in path:
                        thresh = output_dict['detection_scores'][661]
                        my_df.loc['Image12_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_118, thresh]
                    elif '208' in path:
                        thresh = output_dict['detection_scores'][312]
                        my_df.loc['Image13_val'] =[count_10, count_20, count_30, count_40, count_50, count_60, 
                                                  count_70, count_80, count_90, gt_208, thresh]                      
                        
            my_path = '/mnt/work/TESE/Report/Images_ai/count_tables'
            if validation == True:
                my_df.to_csv(my_path + '/' + str(model)+'_val.csv')
            else:
                my_df.to_csv(my_path + '/' + str(model)+'.csv')

Model a070 started


In [ ]:
print(output_dict)